In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA']   = "1"
from os import path as osp
# if 'jupyter' in os.getcwd():
#     os.chdir(osp.join(os.getcwd(), 'masterarbeit', 'code'))
import glob
import time
import argparse
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdmnotebook
import pickle
from pathlib import Path
from typing import Any
from typing import Callable
from typing import Optional
from typing import Tuple
from typing import Union
from typing import Dict
from typing import List
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
mpl.rc('axes', unicode_minus=False)
preamble = r'\usepackage{amsmath}'  # LaTeX preamble command
mpl.rcParams['text.latex.preamble'] = preamble

# import seaborn as sns
import networkx as nx
import math
import numpy as np

import gc

# pytorch imports
import torch
from torch import Tensor, nn, cuda
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.cuda.amp import GradScaler, autocast

# pytorch geometric imports
import torch_geometric
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Compose

# lightning imports
from lightning.pytorch.utilities.combined_loader import CombinedLoader

from IPython.display import display, clear_output

from google.cloud import storage

import sys
# Add the 'code' directory to sys.path to make the  submodules available
# sys.path.append('/home/jupyter/masterarbeit/code')

from util.utils import generate_log_name
from util.plot_utils import *

import logging as log

from data.dataset.GraphDataset import GraphDataset

from model.transform import CollapseChannels, ExtractSquare
from model.autoencoder import Autoencoder
from model.predictor import Predictor
from model.DAN import GradientReversalLayer, DomainDiscriminator

from model.criterions import WeightedMSELoss, MSLELoss, FocalLoss, ZeroInflatedLoss, CustomHuberLoss

from fullmodel import Model

In [2]:
exp_id='exp08'

In [3]:
NUM_WORKERS: int = 0
BATCH_SIZE: int = 64
NUM_CHANNELS: int = 2
WDW_LENGTH: list = [12, 6]

# Constants that I may change a bit during testing
tgt: str = 'MELBOURNE'
src_list: list = ['ANTWERP']
# src_list: list = ['ANTWERP', 'BANGKOK', 'BARCELONA', 'BERLIN', 'CHICAGO', 'ISTANBUL', 'MOSCOW'] # 7 cities
EPOCHS: int = 2
tgt_data_limit: int = 1680
src_data_limit: int = None
LOGGING: int = 1

if src_data_limit == -1:
    src_data_limit = None

# Get data from bucket
bucket_name = 'cloud-ai-platform-054ad037-69b6-4c4d-94a1-75d2591213c7'
bucket_folder = 'data/graphs'
local_folder  = 'data/graphs'
download_directory(bucket_name, bucket_folder, local_folder)
bucket_folder = 'data/raw'
local_folder  = 'data/raw'
download_directory(bucket_name, bucket_folder, local_folder)

bucket_output = 'output/models/'

# Constants that I don't intend to change much
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TRAIN_VAL_TEST_SPLIT = [0.8, 0.1, 0.1]

pre_transform = Compose([
    CollapseChannels(),
    ExtractSquare(50, 'central'),
])

static_transform = Compose([
    ExtractSquare(50, 'central'),
])

ds_kwargs = {
    'root_dir': 'data/raw',
    'device': device,
    'pre_transform': pre_transform,
    'static_transform': static_transform,
}

# seed generator for DataLoader
torch.manual_seed(2311)

# Create datasets for each city
ds_dict = {}
for city in src_list:
    ds_dict[city] = GraphDataset(
        cities=[city],
        limit=src_data_limit,
        **ds_kwargs,
    )
    
temp_tgt = GraphDataset(
    cities=[tgt],
    limit=None,
    **ds_kwargs,
)

train_tgt, val_tgt, test_tgt = random_split(
    temp_tgt, [0.08, 0.08, 0.84]
)


train_tgt = DataLoader(train_tgt, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)
val_tgt   = DataLoader(  val_tgt, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)
test_tgt  = DataLoader( test_tgt, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
# Split each dataset into training and test sets
train = {}
val   = {}
test  = {}
for city in ds_dict:
    train_ds, val_ds, test_ds = random_split(
        ds_dict[city], TRAIN_VAL_TEST_SPLIT
    )
    train[city] = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)
    val[city]   = DataLoader(  val_ds, batch_size=BATCH_SIZE, shuffle=True,  drop_last=True)
    test[city]  = DataLoader( test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

train[tgt] = train_tgt
val[tgt]   = val_tgt
test[tgt]  = test_tgt

# Create dataloader for offline training with source cities
source_train = {city: train[city] for city in src_list}
source_dataloader = CombinedLoader(source_train, mode='max_size_cycle')

source_test = {city: test[city] for city in src_list}
sourcetest_dataloader = CombinedLoader(source_test, mode='max_size_cycle')

target_dataloader = CombinedLoader({tgt: train[tgt]}, mode='max_size_cycle')
targettest_dataloader = CombinedLoader({tgt: test[tgt]}, mode='max_size_cycle')

# Create dataloader for online training with source and target cities
train_dataloader = CombinedLoader(train, mode='max_size_cycle')

# Create dataloader for validation with source and target cities
val_dataloader = CombinedLoader(val, mode='max_size_cycle')

# Create dataloader for testing with source and target cities
test_dataloader = CombinedLoader(test, mode='max_size_cycle')

In [ ]:
##############################################################################
########################## INSTANTIATING THE MODEL ###########################
##############################################################################
AE_K_CHEB = 3
AE_CONV_DIM = 16
AE_LINEAR_DIM = 8
AE_DROPOUT = 0.5
AE_ACTIVATION = 'tanh'

AE_parameters = {
    'K_cheb': AE_K_CHEB,
    'conv_dim': AE_CONV_DIM,
    'linear_dim': AE_LINEAR_DIM,
    'dropout': AE_DROPOUT,
    'activation': AE_ACTIVATION,
    'num_channels': NUM_CHANNELS,
    'device': device,
}

DD_SEQ_LEN = 12
DD_FEAT_DIM = AE_LINEAR_DIM
DD_LEFT_NODES = 1750
DD_parameters = {
    'seq_len': DD_SEQ_LEN,
    'feat_dim': DD_FEAT_DIM,
    'left_nodes': DD_LEFT_NODES,
}

# autoencoder linear dims + 4 sin-cos time features
PRED_FEATURES   = AE_LINEAR_DIM + 4
PRED_LINEAR_DIM = 32
PRED_PERIODS_IN = 12
PRED_PERIODS_OUT = [0, 1, 2, 5, 8, 11]
PRED_ACTIVATION  = 'relu'

PR_parameters = {
    'features': PRED_FEATURES,
    'linear_dim': PRED_LINEAR_DIM,
    'periods_in': PRED_PERIODS_IN,
    'periods_out': PRED_PERIODS_OUT,
    'activation': PRED_ACTIVATION,
    'num_channels': NUM_CHANNELS,
    'device': device,
    'batch_size': BATCH_SIZE,
    
}
num_epochs = EPOCHS
dataloaders = train_dataloader, target_dataloader, targettest_dataloader
AE_criterion = nn.MSELoss()
PR_criterion = nn.MSELoss()
optimizer_parameters = 5e-4, 5e-5
BATCH_SIZE = BATCH_SIZE
dd_lambda = 0.1

folder = osp.join('training logs', 'models', exp_id)
check_dir(folder)

linear_dims = [8, 16, 32, 64]
test_errors = {}
for i, linear_dim in enumerate(linear_dims):
    PR_parameters['linear_dim'] = linear_dim
    model = Model(
        AE_parameters=AE_parameters,
        DD_parameters=DD_parameters,
        PR_parameters=PR_parameters,
        num_epochs=num_epochs,
        dataloaders=dataloaders,
        AE_criterion=AE_criterion,
        PR_criterion=PR_criterion,
        optimizer_parameters=optimizer_parameters,
        BATCH_SIZE=BATCH_SIZE,
        dd_lambda=dd_lambda,
        folder=folder,
        specs=f'{linear_dim}',
        tgt=tgt,
        val_dl=val_dataloader
    ).to(device)
    
    if i == 0:
        model.ae_train('pretrain', save=False, lambda_update=1.5, plot=1)
        model.ae_train('finetune', save=True, plot=1)
    else:
        model.load_module('autoencoder', folder, 'ae_8.pth')
            
    model.pred_train('pretrain', save=False)
    model.pred_train('finetune', save=True)
    
    errors = model.pred_test()
    test_errors[linear_dim] = errors
    plot_losses_boxplot(
        test_errors, 
        'Linear Dimension', 
        save=False, 
        exp_id=exp_id,
    )

plot_losses_boxplot(
    test_errors, 
    'Linear Dimension', 
    save=True,
    exp_id=exp_id,
)

Epochs:   0%|          | 0/2 [00:00<?,?it/s]

Batches:   0%|          | 0/541 [00:00<?,?it/s]